In [37]:
import collections

import os
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM,GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14422796308512904380
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357302272
locality {
  bus_id: 1
}
incarnation: 12625082302388410089
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
The dataset used in this project to train and evaluate the pipeline is from [WMT](http://www.statmt.org/). I am only using a small vocabulary of the original corpus as of now.

In [4]:
def load_data(path):
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data.split('\n')

english_sentences = load_data('data/small_vocab_en')
french_sentences = load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, it looks like they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  But, the text still requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Here, I'm having a look at the complexity of the dataset I'm working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


## Preprocess

1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

### Tokenize (IMPLEMENTATION)
Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. 

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

# Testing Tokenizer
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

This is done by using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [9]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if not length:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding='post')

# Testing Padding
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline

In [10]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
These are the various neural network architecturesyou I'm experimenting with in this project.
- Model 1 is a simple vanilla RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an Encoder-Decoder RNN
- Model 5 is a Bidirectional RNN with Embedding

### Ids Back to Text
The neural network will be translating the input to words ids.  As, we want the French translation, the function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.

In [11]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

`logits_to_text` function loaded.


### Model 1: Vanilla RNN
A basic RNN model is a good baseline for sequence data.

In [13]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    input_sequence = Input(shape=(input_shape[1:]))
    rnn = GRU(english_vocab_size, return_sequences=True)(input_sequence)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    predictions = Activation('softmax')(logits)
    model = Model(inputs=input_sequence, outputs=predictions)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=0.001),
                  metrics=['accuracy'])
    return model

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 12s 109us/step - loss: 2.7112 - acc: 0.4608 - val_loss: nan - val_acc: 0.5316
Epoch 2/10
110288/110288 [==============================] - 10s 91us/step - loss: 1.7717 - acc: 0.5678 - val_loss: nan - val_acc: 0.5933
Epoch 3/10
110288/110288 [==============================] - 10s 91us/step - loss: 1.5147 - acc: 0.6023 - val_loss: nan - val_acc: 0.6106
Epoch 4/10
110288/110288 [==============================] - 10s 91us/step - loss: 1.3909 - acc: 0.6199 - val_loss: nan - val_acc: 0.6298
Epoch 5/10
110288/110288 [==============================] - 10s 91us/step - loss: 1.3019 - acc: 0.6347 - val_loss: nan - val_acc: 0.6408
Epoch 6/10
110288/110288 [==============================] - 10s 91us/step - loss: 1.2284 - acc: 0.6468 - val_loss: nan - val_acc: 0.6548
Epoch 7/10
110288/110288 [==============================] - 10s 91us/step - loss: 1.1698 - acc: 0.6563 - val_loss: nan - val_a

### Model 2: Embedding
An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors. This model creates a RNN model using embedding.

In [28]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    input_sequence = Input(shape=(input_shape[1],))
    embedding_sequence = Embedding(
        input_dim=english_vocab_size,
        output_dim=128,
        input_length=input_shape[1:][0])(input_sequence)
    rnn = GRU(200, return_sequences=True)(embedding_sequence)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    predictions = Activation('softmax')(logits)
    
    model = Model(inputs=input_sequence, outputs=predictions)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=0.001),
                  metrics=['accuracy'])
    return model

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size + 1,
    french_vocab_size + 1)
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 12s 112us/step - loss: 3.2397 - acc: 0.4178 - val_loss: 2.5230 - val_acc: 0.4822
Epoch 2/10
110288/110288 [==============================] - 12s 107us/step - loss: 2.0147 - acc: 0.5575 - val_loss: 1.5297 - val_acc: 0.6386
Epoch 3/10
110288/110288 [==============================] - 12s 107us/step - loss: 1.2880 - acc: 0.6902 - val_loss: 1.0872 - val_acc: 0.7403
Epoch 4/10
110288/110288 [==============================] - 12s 107us/step - loss: 0.9311 - acc: 0.7682 - val_loss: 0.7898 - val_acc: 0.7919
Epoch 5/10
110288/110288 [==============================] - 12s 107us/step - loss: 0.6953 - acc: 0.8114 - val_loss: 0.6154 - val_acc: 0.8283
Epoch 6/10
110288/110288 [==============================] - 12s 107us/step - loss: 0.5654 - acc: 0.8382 - val_loss: 0.5180 - val_acc: 0.8494
Epoch 7/10
110288/110288 [==============================] - 12s 107us/step - loss: 0.4805 - acc: 0.8589

### Model 3: Bidirectional RNNs
The restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [34]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    input_sequence = Input(input_shape[1:])
    rnn = Bidirectional(GRU(english_vocab_size, return_sequences=True))(input_sequence)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(rnn)
    model = Model(inputs=input_sequence, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=0.001),
                  metrics=['accuracy'])
    return model
    
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

bd_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 17s 150us/step - loss: 2.2795 - acc: 0.5364 - val_loss: nan - val_acc: 0.6058
Epoch 2/10
110288/110288 [==============================] - 16s 143us/step - loss: 1.4122 - acc: 0.6195 - val_loss: nan - val_acc: 0.6325
Epoch 3/10
110288/110288 [==============================] - 16s 143us/step - loss: 1.2524 - acc: 0.6445 - val_loss: nan - val_acc: 0.6548
Epoch 4/10
110288/110288 [==============================] - 16s 143us/step - loss: 1.1586 - acc: 0.6647 - val_loss: nan - val_acc: 0.6745
Epoch 5/10
110288/110288 [==============================] - 16s 143us/step - loss: 1.0864 - acc: 0.6793 - val_loss: nan - val_acc: 0.6824
Epoch 6/10
110288/110288 [==============================] - 16s 143us/step - loss: 1.0294 - acc: 0.6883 - val_loss: nan - val_acc: 0.6910
Epoch 7/10
110288/110288 [==============================] - 16s 143us/step - loss: 0.9859 - acc: 0.6949 - val_loss: nan -

### Model 4: Encoder-Decoder
This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

In [38]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    input_sequence = Input(input_shape[1:])
    encoder = LSTM(english_vocab_size)(input_sequence)
    decoder = RepeatVector(output_sequence_length)(encoder)
    decoder = LSTM(english_vocab_size, return_sequences=True)(decoder)
    decoded = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(decoder)

    model = Model(inputs=input_sequence, outputs=decoded)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(lr=0.001),
                  metrics=['accuracy'])

    return model
    
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))


encdec_rnn_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 22s 200us/step - loss: 2.9418 - acc: 0.4421 - val_loss: nan - val_acc: 0.4837
Epoch 2/10
110288/110288 [==============================] - 21s 192us/step - loss: 2.2290 - acc: 0.5128 - val_loss: nan - val_acc: 0.5320
Epoch 3/10
110288/110288 [==============================] - 21s 192us/step - loss: 1.9061 - acc: 0.5444 - val_loss: nan - val_acc: 0.5615
Epoch 4/10
110288/110288 [==============================] - 21s 192us/step - loss: 1.6598 - acc: 0.5686 - val_loss: nan - val_acc: 0.5732
Epoch 5/10
110288/110288 [==============================] - 21s 192us/step - loss: 1.5520 - acc: 0.5849 - val_loss: nan - val_acc: 0.5943
Epoch 6/10
110288/110288 [==============================] - 21s 192us/step - loss: 1.4683 - acc: 0.6015 - val_loss: nan - val_acc: 0.6095
Epoch 7/10
110288/110288 [==============================] - 21s 193us/step - loss: 1.4090 - acc: 0.6113 - val_loss: nan -

### Model 5: Bidirectional RNN with Embedding
This model incorporates embedding and a bidirectional rnn into one model.

In [42]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    input_sequence = Input(shape=(input_shape[1:]))
    embed = Embedding(english_vocab_size, 128, input_length=input_shape[1:][0])(input_sequence)
    
    encoder = Bidirectional(GRU(256))(embed)
    dense_layer = Dense(128, activation='relu')(encoder)
    decoding_layer = RepeatVector(output_sequence_length)(dense_layer)
    
    decoder = Bidirectional(GRU(256, return_sequences=True))(decoding_layer)
    preds = TimeDistributed(Dense(french_vocab_size,activation='softmax'))(decoder)
    
    model = Model(inputs=input_sequence, outputs=preds)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=0.005),
                  metrics=['accuracy'])
    return model

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

model_rnn_final = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

model_rnn_final.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(model_rnn_final.predict(tmp_x[:1])[0], french_tokenizer))

Final Model Loaded
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 38s 348us/step - loss: 2.4003 - acc: 0.4863 - val_loss: nan - val_acc: 0.6019
Epoch 2/10
110288/110288 [==============================] - 37s 333us/step - loss: 1.2174 - acc: 0.6656 - val_loss: nan - val_acc: 0.7118
Epoch 3/10
110288/110288 [==============================] - 37s 334us/step - loss: 0.8592 - acc: 0.7500 - val_loss: nan - val_acc: 0.7831
Epoch 4/10
110288/110288 [==============================] - 37s 332us/step - loss: 0.6304 - acc: 0.8114 - val_loss: nan - val_acc: 0.8365
Epoch 5/10
110288/110288 [==============================] - 37s 333us/step - loss: 0.4465 - acc: 0.8663 - val_loss: nan - val_acc: 0.8851
Epoch 6/10
110288/110288 [==============================] - 37s 334us/step - loss: 0.2976 - acc: 0.9148 - val_loss: nan - val_acc: 0.9297
Epoch 7/10
110288/110288 [==============================] - 37s 333us/step - loss: 0.2023 - acc: 0.943

## Prediction

In [50]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    model = model_final(input_shape = x.shape,
                        output_sequence_length = y.shape[1],
                        english_vocab_size = len(x_tk.word_index),
                        french_vocab_size = len(y_tk.word_index))

    model.fit(x, y, batch_size=1024, epochs=20, validation_split=0.2)

    
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 34s 309us/step - loss: 2.2582 - acc: 0.5038 - val_loss: nan - val_acc: 0.6211
Epoch 2/20
110288/110288 [==============================] - 32s 294us/step - loss: 1.1572 - acc: 0.6780 - val_loss: nan - val_acc: 0.7128
Epoch 3/20
110288/110288 [==============================] - 32s 295us/step - loss: 0.8378 - acc: 0.7493 - val_loss: nan - val_acc: 0.7848
Epoch 4/20
110288/110288 [==============================] - 32s 294us/step - loss: 0.5861 - acc: 0.8194 - val_loss: nan - val_acc: 0.8579
Epoch 5/20
110288/110288 [==============================] - 32s 294us/step - loss: 0.3880 - acc: 0.8854 - val_loss: nan - val_acc: 0.9086
Epoch 6/20
110288/110288 [==============================] - 32s 292us/step - loss: 0.2444 - acc: 0.9319 - val_loss: nan - val_acc: 0.9453
Epoch 7/20
110288/110288 [==============================] - 32s 292us/step - loss: 0.1703 - acc: 0.9525 - val_loss: nan -

In [105]:
from keras.models import load_model

def translate_eng_sentence(eng_sen, french_sen=''): 
    loaded_model = load_model("final_model")
    sentence = eng_sen
    sentence = [english_tokenizer.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=preproc_english_sentences.shape[-1], padding='post')
    sentences = np.array([sentence[0], preproc_english_sentences[0]])
    predictions = loaded_model.predict(sentences, len(sentences))
    y_id_to_word = {value: key for key, value in french_tokenizer.word_index.items()}
    y_id_to_word[0] = '<PAD>'
    print('English Sentence    : he saw a old yellow truck')
    print('Model Prediction    :', ' '.join([y_id_to_word[np.argmax(preproc_english_sentences)] for preproc_english_sentences in predictions[0]]).replace('<PAD>',''))
#     print('Expected Translation: Il a vu un vieux camion jaune')